# Test 05 — Text to Speech (Piper)

What this checks:
- Can import `tts_lib` from `common/lib`
- Can synth to a cached wav
- Can play audio via `aplay`
- Queue mode works (optional)

If audio is silent:
- Confirm `aplay -l`
- Try passing a device to `say(..., device='plughw:0,0')`


In [ ]:
# --- Bootstrap: add repo root + common/lib to sys.path ---
from pathlib import Path
import sys

def add_repo_to_path():
    here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if (p / 'lessons').is_dir() and (p / 'common').is_dir():
            repo_root = p
            common_lib = p / 'common' / 'lib'
            if str(repo_root) not in sys.path:
                sys.path.insert(0, str(repo_root))
            if str(common_lib) not in sys.path:
                sys.path.insert(0, str(common_lib))
            print('Repo root:', repo_root)
            print('Added common/lib:', common_lib)
            return repo_root
    raise FileNotFoundError('Could not find repo root (needs lessons/ and common/)')

add_repo_to_path()

In [ ]:
import os
import subprocess

print('Python:', os.popen('python3 -V').read().strip())
print('Piper:', subprocess.getoutput('which piper || true'))
print('aplay:', subprocess.getoutput('which aplay || true'))
print('Voice dir:', os.environ.get('PIPER_VOICE_DIR') or os.path.expanduser('~/.local/share/piper/voices'))

# show audio devices
print('\nAudio devices (aplay -l):')
print(subprocess.getoutput('aplay -l || true'))


In [ ]:
import time
import tts_lib

print('tts_lib loaded:', tts_lib.__file__)
print('VOICE_DIR:', tts_lib.VOICE_DIR)
print('VOICE_MAP keys:', sorted(tts_lib.VOICE_MAP.keys()))

print('Warming piper...')
tts_lib.warm_piper()
print('Warm done')


In [ ]:
# --- Basic blocking speak ---
text = 'Kia ora! This is the Matamoe robot text to speech test.'
path = tts_lib.say(text, voice='ryan', block=True)
print('WAV path:', path)
print('Duration (s):', round(tts_lib.wav_duration_seconds(path), 2))


In [ ]:
# --- Cached synth check (should be fast second time) ---
text = 'Caching test. This should be quicker the second time.'
t0 = time.time()
p1 = tts_lib.pre_synth(text, voice='ryan')
t1 = time.time()
p2 = tts_lib.pre_synth(text, voice='ryan')
t2 = time.time()
print('path1:', p1)
print('path2:', p2)
print('first synth seconds:', round(t1 - t0, 3))
print('cached seconds:', round(t2 - t1, 3))

tts_lib.say('Cached playback now.', voice='ryan', block=True)


In [ ]:
# --- Non-blocking speak ---
print('Starting non-blocking playback...')
tts_lib.say('This should not block. You should see prints while it speaks.', block=False)

for i in range(10):
    print('tick', i)
    time.sleep(0.2)

print('Done')


In [ ]:
# --- Queue test (optional) ---
q = tts_lib.get_tts_queue()
q.schedule('Queue test. First message.', delay_s=0.0, block=True)
q.schedule('Second message after half a second.', delay_s=0.5, block=True)
q.schedule('Third message after one second.', delay_s=1.0, block=True)
print('Queued 3 messages.')


## If you get silence

Try specifying the device:

```python
tts_lib.say('Test on a specific output device', device='plughw:0,0')
```

Use `aplay -l` output above to pick the right one.